In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Paths
INPUT_DIR = "Input_Images"  # Input Image Folder
OUTPUT_DIR = "organized_images"  
CATEGORY_DIR = os.path.join(OUTPUT_DIR, "Classified_Images")  
DATE_DIR = os.path.join(OUTPUT_DIR, "Date_Wise_Images")  
MODEL_PATH = "Trained_Model/thermal_visible_classifier.h5"  # Path to Trained Model
METADATA_CSV = os.path.join(OUTPUT_DIR, "metadata.csv")  # CSV Output File

# Ensure Output Folders Exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CATEGORY_DIR, exist_ok=True)
os.makedirs(DATE_DIR, exist_ok=True)

# Load trained CNN model
if not os.path.exists(MODEL_PATH):
    print(f"❌ Error: Model file '{MODEL_PATH}' not found! Train the model first.")
    exit()

model = load_model(MODEL_PATH)

# Function to extract metadata from an image
def extract_metadata(image_path):
    metadata = {"FileName": os.path.basename(image_path)}
    
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        
        if exif_data:
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                if tag_name == "GPSInfo":
                    gps_data = {GPSTAGS.get(t, t): v for t, v in value.items()}
                    
                    # Extract full latitude & longitude
                    lat, lon = extract_gps_coordinates(gps_data)
                    metadata["Latitude"] = lat
                    metadata["Longitude"] = lon
                elif tag_name == "DateTime":
                    date_part, time_part = format_datetime(value)
                    metadata["Date"] = date_part
                    metadata["Time"] = time_part
    except Exception as e:
        print(f"Error extracting metadata from {image_path}: {e}")
    
    return metadata

# Function to extract full GPS coordinates
def extract_gps_coordinates(gps_data):
    try:
        lat_values = gps_data.get("GPSLatitude", (0, 0, 0))
        lon_values = gps_data.get("GPSLongitude", (0, 0, 0))
        
        lat = lat_values[0] + (lat_values[1] / 60.0) + (lat_values[2] / 3600.0)
        lon = lon_values[0] + (lon_values[1] / 60.0) + (lon_values[2] / 3600.0)

        return f"{lat:.10f}", f"{lon:.10f}"  # Full-precision GPS values
    except:
        return "Unknown", "Unknown"

# Function to format datetime
def format_datetime(datetime_str):
    try:
        date_part, time_part = datetime_str.split(" ")
        formatted_date = date_part.replace(":", "-")  
        return formatted_date, time_part
    except:
        return "Unknown", "Unknown"

# Function to classify the image as Thermal or Visible
def predict_image_category(image_path):
    img_height, img_width = 224, 224  

    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0  
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    return "Visible" if prediction[0][0] > 0.5 else "Thermal"

# Function to process and organize images
def organize_images():
    metadata_list = []  # List to store metadata

    for filename in os.listdir(INPUT_DIR):
        file_path = os.path.join(INPUT_DIR, filename)
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue  

        metadata = extract_metadata(file_path)

        # Extract metadata values
        date = metadata.get("Date", "Unknown")
        time = metadata.get("Time", "Unknown")
        lat = metadata.get("Latitude", "Unknown")
        lon = metadata.get("Longitude", "Unknown")

        # Construct new filename using Date, Latitude, and Longitude
        if date != "Unknown" and lat != "Unknown" and lon != "Unknown":
            new_filename = f"{date}_{lat}_{lon}.jpg"
        else:
            new_filename = f"unknown_{filename}"

        # Classify the image as Thermal or Visible
        category = predict_image_category(file_path)

        # Create category-based folders (Thermal & Visible)
        category_folder = os.path.join(CATEGORY_DIR, category)
        os.makedirs(category_folder, exist_ok=True)

        # Create date-based folder inside Date_Wise_Images/
        date_folder = os.path.join(DATE_DIR, date if date != "Unknown" else "Unknown_Date")
        os.makedirs(date_folder, exist_ok=True)

        # Move and rename file into both classified & date-based folders
        category_path = os.path.join(category_folder, new_filename)
        date_path = os.path.join(date_folder, new_filename)

        shutil.copy(file_path, category_path)  # Store in Thermal/Visible folder
        shutil.move(file_path, date_path)  # Move to Date-wise folder

        # Append metadata to list
        metadata_list.append({
            "Original FileName": filename,
            "New FileName": new_filename,
            "Date": date,
            "Time": time,
            "Latitude": lat,
            "Longitude": lon,
            "Category": category,
            "Category Path": category_path,
            "Date Path": date_path
        })

    # Save metadata to CSV in a structured format
    metadata_df = pd.DataFrame(metadata_list)
    metadata_df.to_csv(METADATA_CSV, index=False)

    print("✅ Metadata extraction and image organization successfully completed!")
    print(f"📄 Metadata saved to {METADATA_CSV}")

# Run the script
organize_images()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━